In [8]:
import pandas as pd
import numpy as np
# !pip install statsmodels
import statsmodels
from statsmodels.stats.proportion import proportion_confint

In [ ]:
root_output_dir = os.environ.get("DART_WORK_DIR", "")

In [4]:
hyena_table = os.path.join(root_output_dir,"task_2_footprinting/outputs/evals/likelihoods/hyenadna-large-1m-seqlen-hf.tsv")
hyena_data = pd.read_csv(hyena_table, sep="\t", index_col=0)

gena_table = os.path.join(root_output_dir,"task_2_footprinting/outputs/evals/likelihoods/gena-lm-bert-large-t2t.tsv")
gena_data = pd.read_csv(gena_table, sep="\t", index_col=0)

dnabert_table = os.path.join(root_output_dir,"task_2_footprinting/outputs/evals/likelihoods/DNABERT-2-117M.tsv")
dnabert_data = pd.read_csv(dnabert_table, sep="\t", index_col=0)

nt_table = os.path.join(root_output_dir,"task_2_footprinting/outputs/evals/likelihoods/nucleotide-transformer-v2-500m-multi-species.tsv")
nt_data = pd.read_csv(nt_table, sep="\t", index_col=0)


In [34]:
def get_cis(val_list, num_trials=200):
    ci_tup_list = []
    for val in val_list:
        ci = proportion_confint(round(val * num_trials), num_trials, method="normal")
        ci_tup_list.append((val, round(ci[0], 3), round(ci[1], 3)))
    return ci_tup_list


In [35]:
ci_agg_list = []
for table in [hyena_data, gena_data, dnabert_data, nt_data]:
    ci_agg_list.append(get_cis(table["Accuracy"].values))

In [39]:
ci_table = pd.DataFrame(ci_agg_list, index=["HyenaDNA", "GENA-LM", "DNABERT-2", "Nucleotide Transformer"], columns=nt_data.index).T



In [41]:
#change from likelihoods to embeddings if necessary
out_file = os.path.join(root_output_dir,"task_2_footprinting/outputs/evals/likelihoods/confidence_intervals.tsv")

ci_table.to_csv(out_file, sep="\t", index=True, header=True)
